In [1]:
import pandas as pd
import numpy as np
from load_data import *
from config import *
from cal_accuracy import *
from sklearn.metrics import r2_score
import h2o
from h2o.automl import H2OAutoML

In [2]:
train_path = r'./input/train_features_100_columns.xlsx'
train_X, train_y = load_train(train_path, columns=used_feature_columns)

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 12 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Black_ncgltp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.959 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [4]:
data = pd.concat([train_X, train_y], axis=1)
data

,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,8_mean,9_mean,10_mean,...,12_skew,13_skew,14_skew,15_skew,16_skew,17_skew,18_skew,19_skew,20_skew,score
0,1.574413,-0.595018,0.677216,-0.696579,0.136810,-0.613404,0.850699,0.431330,0.328712,1.022111,...,-1.102672,0.969074,-0.247969,0.685464,-0.500171,-2.014586,-0.348453,-0.676601,1.076558,5.95492
1,1.244477,-0.137308,-0.301510,-0.482496,0.032674,-1.651424,-1.423365,-0.360956,-0.555029,-0.188058,...,0.682936,1.017500,-1.057441,0.249369,-0.922953,-1.800550,-0.241787,-2.128465,0.254249,4.87705
2,0.931657,0.703500,-1.319554,1.406807,-0.808947,1.479099,-0.308815,1.335414,-0.309134,0.376286,...,-1.208664,-0.461975,1.492302,2.497259,4.810327,0.151155,0.618250,-1.766959,-0.762853,4.53279
3,0.259705,0.926657,-0.428194,2.178997,-2.432780,0.567669,-0.597812,-0.508629,-2.574246,-1.296486,...,0.104821,-2.068631,-0.606832,1.555269,0.124300,0.523478,0.047980,2.915161,2.979695,4.97951
4,1.323155,-0.015586,0.167741,-0.787749,0.175858,-1.127500,-0.175234,-0.205967,-1.033645,-0.075280,...,-0.258667,0.617105,-0.370833,-0.439105,0.754251,-0.789184,-0.109633,-0.220446,-0.705462,6.12295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,-0.461677,1.298373,-1.612240,0.530556,-0.414059,-0.272928,0.732004,-0.384005,0.964341,-0.242277,...,-0.562786,0.518348,-0.032733,0.099830,-0.278905,-0.857440,-0.339690,0.320588,0.707936,5.00000
1144,-0.458822,1.133439,-1.931468,0.577434,-0.817595,-0.309936,0.678030,-0.548255,0.737241,-0.807416,...,0.224051,1.145614,-0.886834,0.894299,0.438648,-0.438553,-0.255705,-0.727307,-0.630789,5.10000
1145,-0.405343,1.286867,-1.940625,0.318612,-0.953067,-0.336904,0.413767,-0.676147,0.841470,-0.597713,...,-0.196911,1.931289,0.189183,1.017778,0.739885,0.209159,-0.355751,0.121028,0.689736,4.80000
1146,-0.124733,1.200254,-1.544272,0.515851,-0.326352,-0.334674,0.675748,-0.490826,0.607516,0.015397,...,-0.392496,0.939524,-1.258123,-0.181848,-0.073388,-0.125529,-0.692759,1.037686,1.232057,4.90000


In [5]:
x = used_feature_columns
y = 'score'

In [6]:
h2o_data = h2o.H2OFrame(data)
# 拆分数据集为训练集和测试集
train, test = h2o_data.split_frame(ratios=[0.9], seed=42)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
# 输出训练集和测试集的形状
print('Train shape:', train.shape)
print('Test shape:', test.shape)

Train shape: (1042, 101)
Test shape: (106, 101)


In [8]:
automl = H2OAutoML(max_runtime_secs=22000)
# 使用AutoML搜索最佳模型
automl.train(x=x, y=y, training_frame=train)

AutoML progress: |█
00:56:40.499: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),37/340
# GBM base models (used / total),6/38
# DeepLearning base models (used / total),31/299
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [9]:
# 查看AutoML搜索结果
lb = automl.leaderboard
print(lb.head())

# 预测测试集
predictions = automl.leader.predict(test)

predictions

model_id                                                    rmse       mse       mae      rmsle    mean_residual_deviance
StackedEnsemble_AllModels_6_AutoML_1_20230329_05640     0.505878  0.255913  0.364264  0.0858118                  0.255913
StackedEnsemble_AllModels_5_AutoML_1_20230329_05640     0.50749   0.257546  0.367305  0.0862244                  0.257546
StackedEnsemble_BestOfFamily_8_AutoML_1_20230329_05640  0.51878   0.269133  0.372219  0.0874731                  0.269133
StackedEnsemble_BestOfFamily_5_AutoML_1_20230329_05640  0.52423   0.274817  0.378479  0.0884451                  0.274817
StackedEnsemble_BestOfFamily_7_AutoML_1_20230329_05640  0.5254    0.276045  0.378637  0.0886705                  0.276045
StackedEnsemble_Best1000_1_AutoML_1_20230329_05640      0.525417  0.276063  0.387519  0.0890639                  0.276063
StackedEnsemble_AllModels_4_AutoML_1_20230329_05640     0.529145  0.279994  0.390549  0.0897857                  0.279994
StackedEnsemble_BestOfFa

predict
4.38703
5.51507
4.8485
4.46082
5.42533
6.07483
4.71611
6.47602
4.07976
4.46628
